In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [76]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd
from IPython.display import display, HTML, Markdown
import seaborn as sns

import util
from util.year import year as current_year
from db import session
import model

In [38]:
evaluations = session.query(
    model.Task,
    func.count(distinct(model.User.id)).label('evals_count'),
).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Task.modules).\
    join(model.Module.evaluations).\
    join(model.Evaluation.r_user)

evaluations_per_task = evaluations.\
    group_by(model.Task).order_by(model.Wave.id, model.Task.id)

evaluations_per_task_d = {
    task: evals_count
    for (task, evals_count) in evaluations_per_task.all()
}

In [39]:
successful_evaluations = evaluations_per_task.\
    filter(model.Evaluation.ok == True)

successful_evaluations_d = {
    task: evals_count
    for (task, evals_count) in successful_evaluations.all()
}

In [47]:
per_module = session.query(
    model.Evaluation.user.label('user_id'),
    func.count(model.Evaluation.id).label('eval_count'),
).\
    join(model.Evaluation.r_module).join(model.Module.r_task).\
    join(model.Task.r_wave).\
    filter(model.Wave.year == current_year.id).\
    group_by(model.Evaluation.user,
             model.Evaluation.module).subquery()

EVAL_LIMITS = [10, 30, 50]
problematic_tasks = {
    limit: {
                task: evals_count
                for (task, evals_count) in (
                    evaluations.
                    filter(model.Evaluation.ok == False).
                    join(per_module, per_module.c.user_id == model.User.id).\
                    group_by(model.Task).\
                    filter(per_module.c.eval_count > limit).all()
                )
        }
    for limit in EVAL_LIMITS
}

In [60]:
all_waves = session.query(model.Wave).\
    filter(model.Wave.year == current_year.id).\
    order_by(model.Wave.index).all()

In [81]:
def show_wave_stats(wave):
    tasks = [
        (
            task,
            evaluations_per_task_d[task],
            successful_evaluations_d[task],
            problematic_tasks[10][task] if task in problematic_tasks[10] else 0,
            problematic_tasks[30][task] if task in problematic_tasks[30] else 0,
            problematic_tasks[50][task] if task in problematic_tasks[50] else 0,
        )
        for task in evaluations_per_task_d
        if task.wave == wave.id
    ]
    tasks.sort(key=lambda x: x[0].id)

    df = pd.DataFrame(tasks, columns=[
        'Task',
        'All Evaluations',
        'Successful Evaluations',
        'More than 10 evaluations',
        'More than 30 evaluations',
        'More than 50 evaluations',
    ])
    df = df.set_index('Task')
    
    cm = sns.light_palette("red", as_cmap=True)
    df = df.style.background_gradient(subset=[
        'More than 50 evaluations',
        'More than 30 evaluations',
        'More than 10 evaluations',
    ], cmap=cm)
    
    display(Markdown('# {name}'.format(name=wave.caption)))
    display(df)


for wave in all_waves:
    show_wave_stats(wave)

# Nultá vlna

,All Evaluations,Successful Evaluations,More than 10 evaluations,More than 30 evaluations,More than 50 evaluations
Task,,,,,
Vítej v KSI,685,685,101,18,4
Vaření proměnných,511,500,190,30,8
Seznámení s pythonem,442,417,133,25,7
Složitost,422,413,120,20,5
Korektnost,398,370,134,25,5
Úvod do seznamů,352,342,200,42,15
Funkce,286,268,90,20,6
Start pro zkušené,430,429,11,0,0
Asymptotická složitost,291,227,170,39,13


# Hrubá síla

,All Evaluations,Successful Evaluations,More than 10 evaluations,More than 30 evaluations,More than 50 evaluations
Task,,,,,
Rekurze,240,240,82,26,9
Brute-force,243,242,38,6,1
Permutace,216,210,79,20,7
Variace s opakováním,234,233,138,38,11
Variace bez opakování,198,195,54,12,5
Generování podmnožin,181,176,103,28,10
Backtracking,148,145,30,10,3
Problém osmi dam,102,98,27,12,3
Jigsaw Sudoku,54,54,0,0,0


# Rozděl a panuj

,All Evaluations,Successful Evaluations,More than 10 evaluations,More than 30 evaluations,More than 50 evaluations
Task,,,,,
Zmenši a panuj,156,155,5,3,1
Transformuj a panuj,119,110,55,18,6
Rozděl a panuj,128,127,96,37,14
Binární vyhledávání,112,111,63,23,9
Rozděluj a slučuj,60,60,0,0,0
QuickSort,111,111,63,27,10
Rekurzivní volání,107,103,92,37,12
Binárne vyhľadávanie 2.0,76,76,0,0,0
Karlík jede na soustředění,72,72,0,0,0


# Dynamické programování

,All Evaluations,Successful Evaluations,More than 10 evaluations,More than 30 evaluations,More than 50 evaluations
Task,,,,,
Úvod do dynamického programování,105,101,43,19,6
Překrývající se problémy,92,71,77,37,12
Bottom-up,72,72,22,13,5
Rekurentné vzťahy,70,70,35,18,6
Rekonstrukce řešení,59,56,27,15,6
Nejdelší rostoucí podposloupnost,55,54,50,30,10
Batoh,56,56,43,24,8
Mince,51,48,27,16,6
LosKarlosovské součty,43,41,17,9,3


# NP-těžká vlna

,All Evaluations,Successful Evaluations,More than 10 evaluations,More than 30 evaluations,More than 50 evaluations
Task,,,,,
Opakování složitosti,76,76,54,29,11
Složitostní třídy,63,63,46,25,11
NP aneb ověřování řešení,55,53,35,21,7
Redukce,72,52,65,32,9
Trénink redukcí,49,49,9,5,3
O NP-úplných problémoch,44,41,28,19,5
NP hra a řešení složitých problémů,38,38,19,13,4
Strategie pro NP hru,24,20,20,12,4
Tvorba karty,29,29,0,0,0
